In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

def load_and_prepare_data(data):
    df = pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'])
    return df

def get_menu_items(df):
    """Get all possible menu items by category"""
    return df.groupby('Category')['Item'].unique().to_dict()

def train_area_models(df):
    """Train models for each item in specific area"""
    models = {}
    encoders = {}
    
    def create_features(data):
        features = pd.DataFrame()
        features['Year'] = data['Date'].dt.year
        features['Month'] = data['Date'].dt.month
        features['DayOfWeek'] = data['Date'].dt.dayofweek
        features['DayOfMonth'] = data['Date'].dt.day
        return features
    
    # Train a model for each item and delivery type combination
    for item in df['Item'].unique():
        for order_type in ['Takeaway', 'Dining']:  # Limiting to two delivery types
            item_data = df[(df['Item'] == item) & (df['Type of Order'] == order_type)]
            if len(item_data) > 0:
                X = create_features(item_data)
                y = item_data['Orders']
                
                model = RandomForestRegressor(n_estimators=100, random_state=42)
                model.fit(X, y)
                
                models[(item, order_type)] = model
    
    return models

def predict_for_area(df, area, target_date):
    """Predict orders for specific area and date"""
    # Filter data for the specific area
    area_data = df[df['Area'] == area].copy()
    
    if len(area_data) == 0:
        return f"No data available for area: {area}"
    
    # Get all menu items
    menu_items = get_menu_items(df)
    
    # Train models for this area
    models = train_area_models(area_data)
    
    # Prepare prediction date
    pred_date = pd.to_datetime(target_date)
    pred_features = pd.DataFrame({
        'Year': [pred_date.year],
        'Month': [pred_date.month],
        'DayOfWeek': [pred_date.dayofweek],
        'DayOfMonth': [pred_date.day]
    })
    
    # Make predictions for all items and both delivery types
    predictions = []
    for category, items in menu_items.items():
        for item in items:
            for order_type in ['Takeaway', 'Dining']:
                model_key = (item, order_type)
                if model_key in models:
                    prediction = models[model_key].predict(pred_features)[0]
                    if prediction > 0:
                        predictions.append({
                            'Category': category,
                            'Item': item,
                            'Type of Order': order_type,
                            'Predicted_Orders': max(0, round(prediction))
                        })
    
    return pd.DataFrame(predictions)

def display_predictions(predictions):
    """Display predictions in a formatted way"""
    if isinstance(predictions, str):
        print(predictions)
        return
    
    # Sort by Category and Predicted Orders
    predictions_sorted = predictions.sort_values(['Category', 'Predicted_Orders'], 
                                              ascending=[True, False])
    
    # Group by Category
    grouped = predictions_sorted.groupby('Category')
    
    print("\nPredicted Orders by Category:")
    print("-" * 80)
    
    for category, group in grouped:
        print(f"\n{category}:")
        print("-" * 40)
        for _, row in group.iterrows():
            print(f"{row['Item']:<30} {row['Type of Order']:<10} {row['Predicted_Orders']:>4} orders")
    
    print("\nSummary:")
    print("-" * 40)
    print(f"Total predicted orders: {predictions['Predicted_Orders'].sum()}")
    print("\nBy delivery type:")
    print(predictions.groupby('Type of Order')['Predicted_Orders'].sum())

def predict_area_orders(data, area, target_date):
    """Main function to get predictions for specific area and date"""
    # Prepare data
    df = load_and_prepare_data(data)
    
    # Get predictions
    predictions = predict_for_area(df, area, target_date)
    
    # Display results
    display_predictions(predictions)
    
    return predictions

# Example usage
if __name__ == "__main__":
    # Sample data (replace with your actual data)
    data = pd.read_csv('the_burger_spot.csv')
    
    # Specify area and date for prediction
    area = "Neo Tokyo"
    target_date = "2023-11-03"
    
    # Get predictions
    predictions = predict_area_orders(data, area, target_date)